#### Class

In [1]:
class A1:
    # Parent class
    def __init__(self, a, b, c):
        print('A1 init')
        self.a = a
        self.b = b
        self.d = 90
        
        self.__ac = c
        
    def void(self):
        return
    
    def tangible(self):
        return self.void()
    
    def tangible2(self):
        return self.tangible()

In [2]:
class B1(A1):
    # first level sub class
    def __init__(self, a, b, c):
        print('B1 init')
        # call the parent class's init method, can use super() instead
        # super(B1, self).__init__(a, b), or 
        # super().__init__(a, b)
        super().__init__(a, b, c)
        # Strongly hidden from outside, cannot be inherited by sub class
        self.__c = c
        # Hidden from outside
        self._cc = c * c
        
    def void(self):
        return self.a + self.b + self.c
    
    def method1(self, x):
        return self.__c + x
    
    # Property method, can call like an attribute
    @property
    def c(self):
        return self.__c
    
    # Property setter, define method to set the property, specify the property name defined
    # under @property
    @c.setter
    def c(self, c_new):
        self.__c = c_new
        
    # Property deleter, define method to delete the property, call "del instance.c"
    @c.deleter
    def c(self):
        raise TypeError('Cannot delete an attribute.')
    
    # Static method, can call without create an instance, like pd.DataFrame()
    @staticmethod
    def static(x):
        print('A static method, {:}'.format(x))
    
    # Class method, use class itself (not instance) as a parameter
    @classmethod
    def class_method(cls, x):
        print('A classmethod, class name: {}, params: {}'.format(cls.__name__, x))
    
    @classmethod
    def sub_method(cls):
        print('From B1')
        print('Method of {}'.format(cls.__name__))

In [12]:
class B2(A1):
    
    def __init__(self, a, b, c):
        print('B2 init')
        super().__init__(a, b, c)
        self._b2 = a + b
        
    @classmethod
    def sub_method(cls):
        print('From B2')
        print('Method of {}'.format(cls.__name__))

In [44]:
# For multi inheritation, with overwritten init function.
class C2_1(B1, B2):
    
    # cannot set default super_class=C2, as C2 in the parameter haven't been defined yet
    def __init__(self, a, b, c, super_class):
        print('C2_1 init')
        
        # Test for super() method
        super(super_class, self).__init__(a, b, c)

In [40]:
# For multi inheritation, without overwritten init function.
class C2_2(B1, B2):
    pass

In [ ]:
# Mixin class and method

In [5]:
# For mixin method
class C1:
    def __init__(self, c1, c2):
        print('C1 init')
        self.c1 = c1
        self.__c2 = c2
    
    @property
    def c2(self):
        return self.__c2

In [6]:
# Use mixin method
class BC(C1, B1):
    # second level subclass, mixin another parent class C1
    def __init__(self, a, b, c, c1, c2):
        print('BC init')
        # Initialize multiple parent class, explicitly call the parent classes' init functions
        C1.__init__(self, c1, c2)
        B1.__init__(self, a, b, c)
        

In [7]:
# MRO

In [88]:
# To see the MRO result, call the class name, not the instance name
# C2_1.__mro__, or
C2_1.mro()

[__main__.C2_1, __main__.B1, __main__.B2, __main__.A1, object]

In [64]:
# super() in MRO: super will search parent class method in the order provided by MRO
# e.g. MRO is C2, B1, B2, A1, 
# super() in C2, or super(C2, self).method() will search in B1;
# super(B1, self).method() will search in B2;
# super(B2, self).method() will search in A1.

# For __init__(), will have chained init method through the order from MRO, 
# e.g. super(B1, self).__init__(a, b, c) will use the init method in B2

# Examples about the affects of different super class parameters
sc = None
sc_list = [C2_1, B1, B2]
params = [2, 3, 4, sc]
C2_1_container = []

for class_ in sc_list:
    params[-1] = class_
    print('super_class: {:}'.format(class_))
    # use * to unpack a sequence, refer to function_test
    C2_1_container.append(C2_1(*params))

super_class: <class '__main__.C2_1'>
C2_1 init
B1 init
B2 init
A1 init
super_class: <class '__main__.B1'>
C2_1 init
B2 init
A1 init
super_class: <class '__main__.B2'>
C2_1 init
A1 init


In [83]:
# Sub class without an init method, will use the init method from parent
# class, according to order from MRO
c2_2 = C2_2(7, 8, 9)

B1 init
B2 init
A1 init


In [46]:
# Inherit from B1, according to MRO, search for and use the first encountered parent method.
c2_1.sub_method()

From B1
Method of C2_1


In [ ]:
# MRO reference
# from http://www.runoob.com/w3cnote/python-super-detail-intro.html

In [90]:
class A:
    def __init__(self):
        self.n = 2

    def add(self, m):
        # 第四步
        # 来自 D.add 中的 super
        # self == d, self.n == d.n == 5
        print('self is {0} @A.add'.format(self))
        self.n += m
        # d.n == 7


class B(A):
    def __init__(self):
        self.n = 3

    def add(self, m):
        # 第二步
        # 来自 D.add 中的 super
        # self == d, self.n == d.n == 5
        print('self is {0} @B.add'.format(self))
        # 等价于 suepr(B, self).add(m)
        # self 的 MRO 是 [D, B, C, A, object]
        # 从 B 之后的 [C, A, object] 中查找 add 方法
        super().add(m)

        # 第六步
        # d.n = 11
        self.n += 3
        # d.n = 14

class C(A):
    def __init__(self):
        self.n = 4

    def add(self, m):
        # 第三步
        # 来自 B.add 中的 super
        # self == d, self.n == d.n == 5
        print('self is {0} @C.add'.format(self))
        # 等价于 suepr(C, self).add(m)
        # self 的 MRO 是 [D, B, C, A, object]
        # 从 C 之后的 [A, object] 中查找 add 方法
        super().add(m)

        # 第五步
        # d.n = 7
        self.n += 4
        # d.n = 11


class D(B, C):
    def __init__(self):
        self.n = 5

    def add(self, m):
        # 第一步
        print('self is {0} @D.add'.format(self))
        # 等价于 super(D, self).add(m)
        # self 的 MRO 是 [D, B, C, A, object]
        # 从 D 之后的 [B, C, A, object] 中查找 add 方法
        super().add(m)

        # 第七步
        # d.n = 14
        self.n += 5
        # self.n = 19

d = D()
d.add(2)
print(d.n)

self is <__main__.D object at 0x0000026FC73FD240> @D.add
self is <__main__.D object at 0x0000026FC73FD240> @B.add
self is <__main__.D object at 0x0000026FC73FD240> @C.add
self is <__main__.D object at 0x0000026FC73FD240> @A.add
19
